# Calculate sigma in Xarray

## Preamble: Import modules, turn off warnings, etc.

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
%matplotlib inline
%load_ext memory_profiler
from dask.distributed import Client, wait
import inspect
import numpy as np
from pathlib import Path
import xarray as xr
import pandas as pd
import seawater

In [3]:
client = Client(scheduler_file="../scheduler.json")

In [4]:
client.restart()

Client Scheduler: tcp://192.168.0.2:8786 Dashboard: http://192.168.0.2:8787/status,Cluster Workers: 2 Cores: 4 Memory: 16.70 GB


## Load the whole model run into a single grid-aware data set

In [5]:
# data_path = Path("../example-data/ORCA05.L46-KKG36F25H/")

In [6]:
# data_files = (
#     sorted(data_path.glob("ORCA05.L46-KKG36F25H_1m_*_grid_?.nc")) +
#     [data_path / "mesh_mask.nc",
#      data_path / "new_maskglo.nc"]
# )

In [7]:
# list(map(print, data_files));

In [8]:
# from xorca.lib import preprocess_orca

# ds = xr.open_mfdataset(data_files,
#                        preprocess=(lambda ds:
#                                    preprocess_orca(data_path / "mesh_mask.nc", ds)))
# ds = ds.chunk({"t": 1, "z_c": 23, "z_l": 23, "x_c": 180, "x_r": 180})
# ds

## And store to the much faster ZARR format

In [9]:
# %%time
# %memit dsz = ds.to_zarr(store="_tmp_ORCA05.L46-KKG36F25H.zarr/", mode="w");

In [10]:
ds = xr.open_zarr("_tmp_ORCA05.L46-KKG36F25H.zarr/")

In [11]:
ds

<xarray.Dataset>
Dimensions:   (t: 24, x_c: 720, x_r: 720, y_c: 509, y_r: 509, z_c: 46, z_l: 46)
Coordinates:
    depth_c   (z_c) float64 dask.array<shape=(46,), chunksize=(23,)>
    depth_l   (z_l) float64 dask.array<shape=(46,), chunksize=(23,)>
    llat_cc   (y_c, x_c) float32 dask.array<shape=(509, 720), chunksize=(509, 180)>
    llat_cr   (y_c, x_r) float32 dask.array<shape=(509, 720), chunksize=(509, 180)>
    llat_rc   (y_r, x_c) float32 dask.array<shape=(509, 720), chunksize=(509, 180)>
    llat_rr   (y_r, x_r) float32 dask.array<shape=(509, 720), chunksize=(509, 180)>
    llon_cc   (y_c, x_c) float32 dask.array<shape=(509, 720), chunksize=(509, 180)>
    llon_cr   (y_c, x_r) float32 dask.array<shape=(509, 720), chunksize=(509, 180)>
    llon_rc   (y_r, x_c) float32 dask.array<shape=(509, 720), chunksize=(509, 180)>
    llon_rr   (y_r, x_r) float32 dask.array<shape=(509, 720), chunksize=(509, 180)>
  * t         (t) datetime64[ns] 2008-01-16T12:00:00 2008-02-15 ...
  * x_c     

## Calculate Sigma

In [12]:
def sigma_from_z_S_T(z, S, T, pr=0):
    p = z
    sigma = seawater.eos80.pden(S, T, z, pr=pr) - 1000.0
    return sigma

In [13]:
%%memit
z = ds.coords["depth_c"]
S = ds.vosaline.where(ds.tmask != 0)
T = ds.votemper.where(ds.tmask != 0)

peak memory: 112.72 MiB, increment: 1.17 MiB


In [14]:
%%memit
sigma0 = xr.apply_ufunc(
    lambda z, S, T: sigma_from_z_S_T(z, S, T, pr=0),
    z, S, T,
    dask='parallelized', output_dtypes=[float, ])
display(sigma0)

<xarray.DataArray (z_c: 46, t: 24, y_c: 509, x_c: 720)>
dask.array<shape=(46, 24, 509, 720), dtype=float64, chunksize=(23, 1, 509, 180)>
Coordinates:
    depth_c  (z_c) float64 dask.array<shape=(46,), chunksize=(23,)>
  * z_c      (z_c) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
    llat_cc  (y_c, x_c) float32 dask.array<shape=(509, 720), chunksize=(509, 180)>
    llon_cc  (y_c, x_c) float32 dask.array<shape=(509, 720), chunksize=(509, 180)>
  * t        (t) datetime64[ns] 2008-01-16T12:00:00 2008-02-15 ...
  * x_c      (x_c) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * y_c      (y_c) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...

peak memory: 113.24 MiB, increment: 0.36 MiB


In [15]:
%%memit
sigma2 = xr.apply_ufunc(
    lambda z, S, T: sigma_from_z_S_T(z, S, T, pr=2000),
    z, S, T,
    dask='parallelized', output_dtypes=[float, ])
display(sigma2)

<xarray.DataArray (z_c: 46, t: 24, y_c: 509, x_c: 720)>
dask.array<shape=(46, 24, 509, 720), dtype=float64, chunksize=(23, 1, 509, 180)>
Coordinates:
    depth_c  (z_c) float64 dask.array<shape=(46,), chunksize=(23,)>
  * z_c      (z_c) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
    llat_cc  (y_c, x_c) float32 dask.array<shape=(509, 720), chunksize=(509, 180)>
    llon_cc  (y_c, x_c) float32 dask.array<shape=(509, 720), chunksize=(509, 180)>
  * t        (t) datetime64[ns] 2008-01-16T12:00:00 2008-02-15 ...
  * x_c      (x_c) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * y_c      (y_c) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...

peak memory: 113.54 MiB, increment: 0.30 MiB


In [16]:
# %%time
# %memit ds0 = sigma0.to_dataset(name="sigma0").to_zarr(store="sigma0.zarr", mode="w")
# sigma0 = xr.open_zarr("sigma0.zarr").sigma0 

In [17]:
# %%time
# %memit ds2 = sigma2.to_dataset(name="sigma2").to_zarr(store="sigma2.zarr", mode="w")
# sigma2= xr.open_zarr("sigma2.zarr").sigma2

In [18]:
# %%time
# %memit h = sigma0.mean(dim=["x_c", "t"]).compute().plot.contourf("y_c", "depth_c", levels=np.linspace(21.0, 29, 17));

In [19]:
# %%time
# %memit h = sigma2.mean(dim=["x_c", "t"]).compute().plot.contourf("y_c", "depth_c", levels=np.linspace(28.5, 37.5, 19));

## Now, group by sigma

In [20]:
ds["sigma0"] = sigma0
ds["sigma2"] = sigma2

In [21]:
ds

<xarray.Dataset>
Dimensions:   (t: 24, x_c: 720, x_r: 720, y_c: 509, y_r: 509, z_c: 46, z_l: 46)
Coordinates:
    depth_c   (z_c) float64 dask.array<shape=(46,), chunksize=(23,)>
    depth_l   (z_l) float64 dask.array<shape=(46,), chunksize=(23,)>
    llat_cc   (y_c, x_c) float32 dask.array<shape=(509, 720), chunksize=(509, 180)>
    llat_cr   (y_c, x_r) float32 dask.array<shape=(509, 720), chunksize=(509, 180)>
    llat_rc   (y_r, x_c) float32 dask.array<shape=(509, 720), chunksize=(509, 180)>
    llat_rr   (y_r, x_r) float32 dask.array<shape=(509, 720), chunksize=(509, 180)>
    llon_cc   (y_c, x_c) float32 dask.array<shape=(509, 720), chunksize=(509, 180)>
    llon_cr   (y_c, x_r) float32 dask.array<shape=(509, 720), chunksize=(509, 180)>
    llon_rc   (y_r, x_c) float32 dask.array<shape=(509, 720), chunksize=(509, 180)>
    llon_rr   (y_r, x_r) float32 dask.array<shape=(509, 720), chunksize=(509, 180)>
  * t         (t) datetime64[ns] 2008-01-16T12:00:00 2008-02-15 ...
  * x_c     

In [23]:
%%memit
temp_t_sigma2_y = ds.groupby_bins("sigma2", np.linspace(25, 40, 31)).apply(
        lambda x: x
    ).votemper
temp_t_sigma2_y

distributed.client - WARNING - Couldn't gather keys {"('zarr-5c5cb87172fd13718edbf9dd545296e1', 0)": [], "('zarr-5c5cb87172fd13718edbf9dd545296e1', 1)": []}


IndexError: index 7794906 is out of bounds for axis 1 with size 7638063